In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [4]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import ModelCheckpoint

In [14]:
data = pd.read_csv('Sentiment Analysis Dataset 2.csv', skiprows=[8835,535881])
data.shape

(1578612, 4)

In [15]:
data.info

<bound method DataFrame.info of           ItemID  ...                                      SentimentText
0              1  ...                       is so sad for my APL frie...
1              2  ...                     I missed the New Moon trail...
2              3  ...                            omg its already 7:30 :O
3              4  ...            .. Omgaga. Im sooo  im gunna CRy. I'...
4              5  ...           i think mi bf is cheating on me!!!   ...
...          ...  ...                                                ...
1578607  1578623  ...               Zzzzzz.... Finally! Night tweeters! 
1578608  1578624  ...                        Zzzzzzz, sleep well people 
1578609  1578625  ...            ZzzZzZzzzZ... wait no I have homework. 
1578610  1578626  ...      ZzZzzzZZZZzzz meh, what am I doing up again? 
1578611  1578627  ...                       Zzzzzzzzzzzzzzzzzzz, I wish 

[1578612 rows x 4 columns]>

In [16]:
data.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [17]:
data = data[['Sentiment', 'SentimentText']]
data.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [18]:
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.lower())
data['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sentiment'] == 1].size)
print(data[ data['Sentiment'] == 0].size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1580354
1576870


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   10,    5,  256],
       [   0,    0,    0, ...,   69,  775, 1257],
       [   0,    0,    0, ...,   24,  192,  408],
       ...,
       [   0,    0,    0, ...,    1,   17,  656],
       [   0,    0,    0, ...,  171,   29,  105],
       [   0,    0,    0, ...,    0,    1,  110]], dtype=int32)

In [20]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
embedd_dim = 50
output_dim = 80

In [ ]:
def my_model():
  model = Sequential()
  model.add(Embedding(max_features, embedd_dim,input_length = X.shape[1]))
  model.add(LSTM(output_dim, dropout = 0.2, recurrent_dropout = 0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1057670, 40) (1057670, 2)
(520942, 40) (520942, 2)


In [ ]:
model = my_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            100000    
_________________________________________________________________
lstm (LSTM)                  (None, 80)                41920     
_________________________________________________________________
dense (Dense)                (None, 2)                 162       
Total params: 142,082
Trainable params: 142,082
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32
path_model ='sentiment_analysis.h5'

h = model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 1, callbacks=[
                ModelCheckpoint(filepath=path_model),
            ])

33053/33053 [==============================] - 4750s 144ms/step - loss: 0.4717 - accuracy: 0.7731


In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

16233/16233 - 152s - loss: 0.4299 - accuracy: 0.7988
score: 0.43
accuracy: 0.80


In [ ]:
pos_count, neg_count, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
    
    if np.argmax(Y_validate[x]) == 0:
        neg_count += 1
    else:
        pos_count += 1
   
    
print("pos_count:", pos_count)
print("neg_count:", neg_count)
print("pos_correct:", pos_correct)
print("neg_correct:", neg_correct)

print("pos_accuracy:", pos_correct/pos_count*100, "%")
print("neg_accuracy:", neg_correct/neg_count*100, "%")

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model('sentiment_analysis.h5')

In [21]:
# loading tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [2]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [22]:
input_text = "I don't feel so good Mr.Stark"
processed_text = re.sub('[^a-zA-z0-9\s]','',input_text.lower())
seq = tokenizer.texts_to_sequences([processed_text])
text = pad_sequences(seq)
pred = model.predict(text)
if(pred[0][0]>pred[0][1]):
  print("Negative with probability: " + str(pred[0][0]))
if(pred[0][0]<pred[0][1]):
  print("Positive with probability: " + str(pred[0][1]))

Negative with probability: 0.91410327
